# 🚀 Tunix Hackathon - Lightweight Version (PyTorch)

This is a simplified version that:
- ✅ Works without HF_TOKEN (uses public models)
- ✅ Uses PyTorch instead of JAX (easier setup)
- ✅ Simple GRPO implementation
- ✅ Direct dataset loading from HuggingFace

**Perfect for quick experiments on Kaggle!**

## Installation

In [ ]:
# Install packages if not already installed
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

# Install required packages
packages = ["torch", "transformers", "datasets", "accelerate"]
for pkg in packages:
    install_if_needed(pkg)

print("\n✅ All packages ready!")

## Imports

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import random

## 1. Load Model (Public - No Token Required)

In [ ]:
# Using Qwen 0.5B - truly public model, no token needed
# Alternative: google/gemma-2-2b-it (might need token)
MODEL = "Qwen/Qwen2.5-0.5B-Instruct"

print(f"Loading {MODEL}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading {MODEL}: {e}")
    print("\nTrying alternative model: microsoft/Phi-3-mini-4k-instruct")
    MODEL = "microsoft/Phi-3-mini-4k-instruct"
    tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ Alternative model loaded!")

## 2. Load GSM8K Dataset

In [ ]:
print("Downloading GSM8K via HuggingFace...")
ds = load_dataset("openai/gsm8k", "main")
train_data = ds["train"]
test_data = ds["test"]
print(f"✅ Train samples: {len(train_data)}")
print(f"✅ Test samples: {len(test_data)}")

## 3. Prepare Data

In [ ]:
def encode(example):
    q = example["question"]
    answer = example["answer"]
    prompt = f"Question: {q}\nAnswer:"
    
    example["prompt"] = prompt
    example["target"] = answer
    return example

train_data = train_data.map(encode)
print("✅ Data prepared!")

## 4. GRPO Functions

In [ ]:
def generate_answer(prompt, max_new_tokens=64):
    """Generate answer from model with error handling"""
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # Use more stable generation parameters
        with torch.no_grad():  # Disable gradient for generation
            output = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,  # Lower temperature for stability
                top_p=0.9,
                top_k=50,  # Add top_k for additional stability
                pad_token_id=tokenizer.eos_token_id,  # Set pad token
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,  # Prevent repetition
                no_repeat_ngram_size=2
            )
        
        text = tokenizer.decode(output[0], skip_special_tokens=True)
        return text.replace(prompt, "").strip()
    except RuntimeError as e:
        if "inf" in str(e).lower() or "nan" in str(e).lower():
            # Fallback to greedy decoding if sampling fails
            print(f"⚠️ Sampling error, using greedy decoding: {e}")
            try:
                with torch.no_grad():
                    output = model.generate(
                        **inputs,
                        max_new_tokens=max_new_tokens,
                        do_sample=False,  # Greedy decoding
                        pad_token_id=tokenizer.eos_token_id,
                        eos_token_id=tokenizer.eos_token_id
                    )
                text = tokenizer.decode(output[0], skip_special_tokens=True)
                return text.replace(prompt, "").strip()
            except Exception as e2:
                print(f"❌ Greedy decoding also failed: {e2}")
                return "[Generation Error]"
        else:
            raise  # Re-raise if it's a different error
    except Exception as e:
        print(f"❌ Generation error: {e}")
        return "[Generation Error]"

def reward_fn(pred, gold):
    """Simple reward: 1 if correct answer number is in prediction"""
    if pred == "[Generation Error]":
        return 0.0  # No reward for errors
    gold_num = gold.split("####")[-1].strip()
    return 1.0 if gold_num in pred else 0.0

## 5. GRPO Training (Simplified)

In [ ]:
# Quick model health check
print("Testing model generation...")
test_prompt = "Question: What is 2+2?\nAnswer:"
try:
    test_output = generate_answer(test_prompt, max_new_tokens=20)
    print(f"✅ Model test successful: {test_output[:50]}...")
except Exception as e:
    print(f"❌ Model test failed: {e}")
    print("Please check model loading or try a different model.")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6)
EPOCHS = 1
GROUP = 2   # Number of samples per step
STEPS = 200  # Number of training steps

print(f"Starting GRPO training...")
print(f"Epochs: {EPOCHS}, Steps: {STEPS}, Group size: {GROUP}")

model.train()  # Set to training mode

for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch+1} ===")
    
    for idx in range(STEPS):
        # Sample random batch
        batch = [train_data[random.randint(0, len(train_data)-1)] for _ in range(GROUP)]
        
        prompts = [b["prompt"] for b in batch]
        golds   = [b["target"] for b in batch]
        
        # Generate predictions
        preds = [generate_answer(p) for p in prompts]
        rewards = [reward_fn(preds[i], golds[i]) for i in range(GROUP)]
        
        # Calculate advantage relative to group average
        avg_reward = sum(rewards) / GROUP
        advantages = [r - avg_reward for r in rewards]
        
        # Calculate losses - FIXED: use proper loss calculation
        losses = []
        for i in range(GROUP):
            # Tokenize prompt
            inp = tokenizer(prompts[i], return_tensors="pt").to(model.device)
            
            # Tokenize generated prediction as target
            pred_tokens = tokenizer(preds[i], return_tensors="pt", add_special_tokens=False).to(model.device)
            
            # Concatenate prompt and prediction for full sequence
            full_input_ids = torch.cat([inp["input_ids"], pred_tokens["input_ids"]], dim=1)
            
            # Create labels (only compute loss on generated part)
            labels = full_input_ids.clone()
            labels[:, :inp["input_ids"].shape[1]] = -100  # Ignore prompt in loss
            
            # Forward pass
            outputs = model(input_ids=full_input_ids, labels=labels)
            logits = outputs.loss
            
            # Weight by advantage (GRPO)
            losses.append(logits * advantages[i])
        
        # Backward pass
        loss = sum(losses) / GROUP
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"⚠️ Step {idx}: Invalid loss, skipping...")
            optimizer.zero_grad()
            continue
            
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        optimizer.zero_grad()
        
        if idx % 20 == 0:
            print(f"Step {idx}, Loss = {loss.item():.4f}, Rewards = {rewards}, Avg Reward = {avg_reward:.2f}")

print("\n🎉 Training Done!")
model.eval()  # Set back to eval mode

## 6. Test on Sample

In [ ]:
# Test on a sample
sample = test_data[0]
prompt = f"Question: {sample['question']}\nAnswer:"
prediction = generate_answer(prompt)

print("Question:", sample['question'])
print("\nPrediction:", prediction)
print("\nGround Truth:", sample['answer'])